# Introduction

This notebook introduces some of the functionality of the *stable-dracor-client* and the *dracor-sandbox* Docker container. It is assumed that the infrastructure has been started by using the docker compose file `compose.yml` (`docker compose up`) and run in the bundled jupyter lab instance on http://localhost:8888.

## DraCor in Docker and the *dracor-sandbox* Docker container

There are several possibilities to set up a local DraCor infrastructure. 

### DraCor Docker containers running on the local machine
One way would be to simply clone the repository of the DraCor API eXist-DB application (https://github.com/dracor-org/dracor-api) and use the provided docker compose file [`compose.yml`](https://github.com/dracor-org/dracor-api/blob/main/compose.yml) to start a local Docker-based DraCor system. If everything went well issuing the the command `docker ps` in a terminal will show multiple running Docker containers. 

(TODO: add screenshot here)

In this approach the environment in which the local infrastructure is running can vary. Depending on the setup of the local host machine (operation-system, version of Docker daemon installed, ...) there might be some issues, e.g. problems connection to `localhost` on some Windows installations. Although, the *stable-dracor-client* can be used with this setup as well, hence we will assume that the following *Docker in Docker* approach to setting up the local infrastructure will be used.

### Docker in Docker
Another way to setup a local DraCor infrastructure using Docker is to make use of a technique that is called *Docker in Docker*. This means that on the host machine there is running a container that has Docker installed. This Docker container is then used to start additional containers inside this controlled environment. This way the environment in which the DraCor services will be executed can be prepared in advanced to make sure that everything works as expected.

When issuing the command `docker compose up` in the root directory of the cloned stable-dracor repository only a single Docker container `dracor-sandbox` will be started. It not only has the Docker daemon installed in it but also features a (this) Jupyter Lab instance (running on http://localhost:8888) that can be used to prepare custom corpora or run Python scripts.

When starting the `dracor-sandbox` container there are no Docker containers running. To test this, it is possible to "enter" the container from the host machine with the command `docker exec -it dracor-sandbox /bin/bash`. In the interactive shell that is now executed inside the container the command `docker ps` will return an empty list:

```
194dc81eb130:/home/dracor# docker ps
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
```

The `dracor-sandbox` container can be left with the command `exit`.

There are also two folders mounted to the container: `import` and `export`. They also show up in the *File Browser* in Jupyter Lab. These folder provide an easy way to get data into and out of the `dracor-sandbox` container. A usecase would be to add locally available XML files. When copying them into the `import` folder on the host machine, they become available to the container. The `export` folder functions the same way. Files that should become available on the host machine even when the Docker container is not running anymore can be stored in this folder. This could, for example, be the results of an analysis or a *manifest* that describes the composition of the custom corpora (see Section X).

Having only one Docker container with the DraCor system components inside also makes it possible to "freeze" not only the whole local Dracor system with the corpora set up (as we prototyped for the paper 
[Detecting Small Worlds in a Corpus of Thousands of Theater Plays](https://github.com/dracor-org/small-world-paper/tree/publication-version)) but in addition to that the environment that was used to assemble it (this Jupyter Lab instance with the Python notebooks).

## Using the *stable-dracor-client*

The following sections showcase some of the features of the *stable-dracor-client*.  

### Installation

Within this Jupyter Lab instance the client has already been installed. If the client should be used standalone it can be installed from within the root directory of the stable-dracor repository (containing the necessary `pyproject.toml` file) with the command `pip install .` (the dot is necessary!).

### Activate Logging
It is recommended to activate logging in the notebook. This can be done by importing the library logging and setting the log level. For normal use the log level `INFO` is sufficient, but if something does not work as expected, the log level should be set to `DEBUG` to see additional messages which might help to identify the problem.
The following cell activates logging for this notebook:

In [ ]:
import logging
#logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)

### Importing the client 
After the client has been installed it can be imported with the following command:

In [ ]:
from stabledracor.client import StableDraCor

### First Step in setting up the infrastructure
To use the client it is necessary to create an instance of the class `StableDraCor` that has been imported to the notebook with the previous command. The most basic command just instantiates an objec without any additional arguments:

In [ ]:
dracor = StableDraCor()

If logging is activated some warnings will appear: We find out that there is no API currently available under the default endpoint url and that no DraCor Docker containers can be found. 

Normally, the output of the client can be trusted in this regard, but to verify that there are no running containers it is possible to directly query the docker daemon running in the `dracor-sandbox` container. The command to list running containers is `docker ps`. In the following cell starts with a `!` though. This is a way to execute shell commands directly from within a Jupyter notebook.

In [ ]:
!docker ps

### GitHub Access Token
The log output from above contains the warning, that a "Personal GitHub Acces Token" has not been supplied. Providing a token is recommended, because for some operations, e.g. adding a corpus from a repository on GitHub, the client relies on the GitHub API. Unauthorized calls to this API are subject to rate limiting, which means, that only a few calls can be send without being authorized to GitHub. The [FAQ](03_faq.ipynb#Why-do-I-need-a-GitHub-Access-Token-and-how-can-I-get-one?) contains instructions  on how to generate such a token and explains how to make it available to a notebook or script as an environmental variable upon upon the creation of a `dracor-sandbox` Docker container. Although it is possible to add the value of the token directly in the script, it is not considered a good practise because an access token is like a password. Especially, if a notebook is shared, this means of "hiding" the password should be used. 

The code in the followin cell uses the library `os` to read an environmental variable `GITHUB_TOKEN` and stores it in the variable `github_token` to later use it when initializing the client.

In [ ]:
import os
github_token = os.environ.get("GITHUB_TOKEN")

In [ ]:
# Optional: Check if the Token is available (see FAQ Notebook)
assert github_token is not None, "It is recommended to use a GitHub Personal Access Token. See FAQ Notebook for details."

If the container has been started without setting a token with the `.env` file as described in the [FAQ](03_faq.ipynb#Why-do-I-need-a-GitHub-Access-Token-and-how-can-I-get-one?), the following command can be used to create it (substitute `value` with your token, of course):

`export GITHUB_TOKEN=value`

It would be possible to issue this command from within the notebook using the prefix `!`, but then the cell containing the value of the token must be removed afterwards for security reasons. 

Another option would be to enter the `dracor-sandbox` container (see [FAQ](03_faq.ipynb#How-can-I-get-into-the-dracor-sandbox-Docker-container?)) or use the Terminal of Jupyter Lab (in the *Launcher* at http://localhost:8888 click on *Terminal*). The shell command `printenv` can be used to list all set variables, `echo $GITHUB_TOKEN` will output the value of the environment variable.

An instance of the client can be created by supplying the token as the argument `github_access_token`:

In [ ]:
dracor = StableDraCor(github_access_token=github_token)

### Attaching Metadata to the *stable-dracor* instance
It is possible to attach to the stable-dracor system. Currently, adding a `name` and a `description` is supported. In addition to that every time an *stable-dracor* is initialized, a universal unique id is generated. This ensures, that each instance can be identified by its unique identifier. These information, `id`, `name` and `description` are also included with the so-called manifest describing a stable-dracor instance. In the following cell we create the an instance once more, provide a GitHub Token and attach metadata:

In [ ]:
dracor = StableDraCor(
    name="my-stable-dracor",
    description="DraCor system created with the introduction notebook to showcase the features of the stable-dracor-client.",
    github_access_token=github_token)

### Additional options when creating a stable-dracor instance
There are some options to change the behaviour of the client instance, e.g. explicitly set a different base url of the API. For details see the [FAQ](03_faq.ipynb#How-can-I-change-the-URL-of-the-API-used).

### Starting the Docker containers
The second step to setup a DraCor system with the *stable-dracor-client* is to start the Docker containers of the system components: eXist-DB with pre-installed application ("DraCor API"), Metrics Service, Frontend, Triple Store. When using the `dracor-sandbox` these services will run inside the "outer" Docker container. 

The most basic way to start these containers is to call the method `run` without any additional arguments: `dracor.run()`. In this case the client will fetch the [default configuration](https://raw.githubusercontent.com/dracor-org/stabledracor/master/configurations/compose.fullstack.empty.yml) which is defined in `compose.fullstack.empty.yml` in the stable-dracor repository on GitHub. 

A configuration file, which is basically a docker-compose file, will specify the Docker images and tags of the components to be used, e.g. currently the default configuration uses the api image `dracor/dracor-api:v0.90.1-local` ([Link](https://github.com/dracor-org/stabledracor/blob/2dc461e6f3d8106f5291ba0b1f6779b7adb52c5d/configurations/compose.fullstack.empty.yml#L8)). 

The images of all DraCor components can be found on [DockerHub](https://hub.docker.com/u/dracor). For example the available images of the DraCor eXist-DB and the application powering the API can be found [here](https://hub.docker.com/r/dracor/dracor-api/tags). The image, that is currently used in the default configuration can be viewed [here](https://hub.docker.com/layers/dracor/dracor-api/v0.90.1-local/images/sha256-e2af569e41398b2b5b527dbb21ada90dea467568ce901b2607b1cc41a6743a75?context=explore). 

Executing `run()` without explicitly specifying a configuration can be considered a fall-back. It is possible to pass a path to a different docker-compose file when starting the client as the keyword argument `compose_file`. Another option is to use `url` and point to a location from which a docker-compose file can be downloaded.

**TODO**: Add configurations to dracor-sandbox container! `!ls ../configurations`! Explain which configuration to use. Difficult because of ARM/AMD architecture?

In [ ]:
dracor.run()

### List running containers
It is possible to check if Docker containers are currently running with the already familiar command `docker ps`. This should now list four running containers.

In [ ]:
!docker ps

The *stable-dracor-client* provides a method (`list_docker_containers`) that gets the same result from the docker daemon and turns it into a data structure native to Python:

In [ ]:
dracor.list_docker_containers()

It is possible to operate on the returned list, e.g. count the number of containers:

In [ ]:
print(f"There are {len(dracor.list_docker_containers())} running Docker containers.")

### Getting API info
A quick way to test if a connection to the API can be established, is calling the method `get_api_info`. This will render the response of the `/info` endpoint http://localhost:8088/api/info of the DraCor API.

In [ ]:
dracor.get_api_info()

### Accessing the front-end
The DraCor frontend running inside the `dracor-sandbox` can be easily accessed from outside the container: Pointing the browser to http://localhost:8088 and/or at http://127.0.0.1:8088 should show the frontend. No corpora have been loaded yet.

The other components of the DraCor system, e.g. the Triple Store, can not be reached from outside the `dracor-sandbox`. See [FAQ](03_faq.ipynb#How-can-I-access-other-services-than-the-frontend-and-the-API-from-outside-the-dracor-sandbox?) on how to map the necessary ports to access the other services if needed.

In [ ]:
dracor.get_manifest()

In [ ]:
dracor.list_docker_images()
#dracor.list_docker_containers()